In [1]:
import pandas as pd
import numpy as np

# from sklearn.datasets import fetch_california_housing

from evidently import ColumnMapping

from evidently.report import Report
from evidently.metrics.base_metric import generate_column_metrics
from evidently.metric_preset import DataDriftPreset, TargetDriftPreset, DataQualityPreset, RegressionPreset
from evidently.metrics import *

from evidently.test_suite import TestSuite
from evidently.tests.base_test import generate_column_tests
from evidently.test_preset import DataStabilityTestPreset, NoTargetPerformanceTestPreset, RegressionTestPreset
from evidently.tests import *

In [2]:
# Charger les données
data_train = pd.read_csv("app_train.csv")
data_test = pd.read_csv("app_test.csv")

In [5]:
# # Charger les données
# data_train = pd.read_csv("app_train.csv")
# data_test = pd.read_csv("app_test.csv")

# Supprimez ces lignes si elles ne sont pas nécessaires
# Elles désactivent simplement les avertissements de pandas
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# Utilisez votre DataFrame client (client_data) à la place de la charge de données California Housing
client_data = data_train.copy()  # Assurez-vous d'avoir une copie de vos données

# Calculer l'âge à partir de la colonne "DAYS_BIRTH" dans le DataFrame client
client_data['Age'] = -client_data['DAYS_BIRTH'] // 365
# Calculer l'âge à partir de la colonne "DAYS_BIRTH" dans le DataFrame client
client_data['Age'] = -client_data['DAYS_BIRTH'] // 365

# Sélectionner les colonnes spécifiques
# selected_columns = ['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT']
selected_columns = ['SK_ID_CURR', 'TARGET', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT','Age']
# Créer un DataFrame avec les colonnes spécifiques
df_cleaned = client_data[selected_columns]

# Sélectionner uniquement les colonnes numériques
numeric_columns = df_cleaned.select_dtypes(include=['number'])

# Créer un nouveau DataFrame avec les colonnes numériques
client_data = df_cleaned[numeric_columns.columns]

# Assurez-vous que les colonnes correspondent à celles de votre DataFrame client
client_data.rename(columns={'TARGET': 'target'}, inplace=True)

# Remplacez les valeurs NaN par des zéros dans la colonne 'target' du dataframe de référence
client_data['target'].fillna(0, inplace=True)

# Ajoutez une colonne de prédiction basée sur la colonne 'target' avec un bruit aléatoire
client_data['prediction'] = client_data['target'] + np.random.normal(0, 5, client_data.shape[0])

client_data['prediction'].fillna(0, inplace=True)

# Créez un DataFrame de référence et actuel à partir de votre DataFrame client
reference = client_data.sample(n=5000, replace=False)
current = client_data.sample(n=5000, replace=False)

# Affichez les DataFrames de référence et actuels
print("DataFrame de Référence (Reference):")
print(reference)
print("\nDataFrame Actuel (Current):")
print(current)
reference = reference.astype(float)
current = current.astype(float)
# Supprimer les lignes contenant des valeurs nulles
reference = reference.dropna()
current = current.dropna()


DataFrame de Référence (Reference):
      SK_ID_CURR  target  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  Age  \
6740      107866       1             0           90000.0    521280.0   32   
7719      109004       0             0          157500.0    263686.5   32   
720       100832       0             1           90000.0    131922.0   41   
4550      105320       0             0          171000.0    765000.0   44   
4434      105183       0             0          180000.0    147888.0   30   
...          ...     ...           ...               ...         ...  ...   
7889      109203       0             0          112500.0    270126.0   38   
2641      103082       0             1           67500.0    170640.0   34   
3805      104447       0             0          157500.0    343800.0   31   
7480      108722       0             0          193500.0    203760.0   47   
9597      111154       0             2          180000.0    254700.0   34   

      prediction  
6740   -5.122973  
7

In [6]:
# Définition du rapport initial avec des métriques de référence
report = Report(metrics=[
    DataDriftPreset(),  # Utilise la métrique de dérive de données prédéfinie
])

# Exécution du rapport avec des données de référence et de données actuelles
report.run(reference_data=reference, current_data=current)

# Affichage du rapport
report

# Création d'un rapport avec des métriques spécifiques sur la colonne 'AveRooms'
report = Report(metrics=[
    ColumnSummaryMetric(column_name='AMT_INCOME_TOTAL'),  # Résume les statistiques de la colonne
    ColumnQuantileMetric(column_name='AMT_INCOME_TOTAL', quantile=0.25),  # Calcule le quantile de la colonne
    ColumnDriftMetric(column_name='AMT_INCOME_TOTAL'),  # Calcule la dérive de la colonne
])

# Exécution du rapport avec des données de référence et de données actuelles
report.run(reference_data=reference, current_data=current)

# Affichage du rapport
report

# Création d'un rapport avec des métriques générées pour plusieurs colonnes
report = Report(metrics=[
    generate_column_metrics(ColumnQuantileMetric, parameters={'quantile': 0.25}, columns=['AMT_INCOME_TOTAL', 'AMT_CREDIT']),
    # Génère des métriques de quantile pour les colonnes spécifiées
])

# Exécution du rapport avec des données de référence et de données actuelles
report.run(reference_data=reference, current_data=current)

# Affichage du rapport
report

# Création d'un rapport avec une variété de métriques pour différentes colonnes et une métrique prédéfinie
report = Report(metrics=[
    ColumnSummaryMetric(column_name='AMT_INCOME_TOTAL'),  # Résume les statistiques de la colonne
    generate_column_metrics(ColumnQuantileMetric, parameters={'quantile': 0.25}, columns='num'),  # Génère des métriques de quantile pour 'num'
    DataDriftPreset(),  # Utilise la métrique de dérive de données prédéfinie
])

# Exécution du rapport avec des données de référence et de données actuelles
report.run(reference_data=reference, current_data=current)

# Affichage du rapport
report

# Conversion du rapport en dictionnaire
report.as_dict()

# Conversion du rapport en format JSON
report.json()

# Explication du code :
# Ce code Python utilise la bibliothèque "datadrift" pour créer des rapports de dérive de données entre un jeu de données de référence (reference) et un jeu de données actuel (current).
# Le code commence par importer les bibliothèques nécessaires.
# Il définit ensuite plusieurs rapports avec différentes métriques pour évaluer la dérive de données, telles que la dérive globale des données, les statistiques de colonnes, les quantiles, etc.
# Chaque rapport est exécuté avec les données de référence et les données actuelles, puis le rapport est affiché.
# À la fin, le code montre comment convertir le rapport en dictionnaire et en format JSON.
# Ces rapports sont utiles pour surveiller et analyser la stabilité des données au fil du temps, ce qui est essentiel dans de nombreuses applications, notamment pour détecter des anomalies ou des changements dans les données.


'{"version": "0.4.8", "metrics": [{"metric": "ColumnSummaryMetric", "result": {"column_name": "AMT_INCOME_TOTAL", "column_type": "num", "reference_characteristics": {"number_of_rows": 5000, "count": 5000, "missing": 0, "missing_percentage": 0.0, "mean": 166293.14, "std": 92130.58, "min": 25650.0, "p25": 112500.0, "p50": 144000.0, "p75": 202500.0, "max": 1350000.0, "unique": 207, "unique_percentage": 4.14, "infinite_count": 0, "infinite_percentage": 0.0, "most_common": 135000.0, "most_common_percentage": 11.18}, "current_characteristics": {"number_of_rows": 5000, "count": 5000, "missing": 0, "missing_percentage": 0.0, "mean": 166710.75, "std": 90994.84, "min": 27000.0, "p25": 112500.0, "p50": 144000.0, "p75": 202500.0, "max": 1350000.0, "unique": 215, "unique_percentage": 4.3, "infinite_count": 0, "infinite_percentage": 0.0, "most_common": 135000.0, "most_common_percentage": 11.2}}}, {"metric": "ColumnQuantileMetric", "result": {"column_name": "CNT_CHILDREN", "column_type": "num", "quan

In [7]:
# Test Suite
tests = TestSuite(tests=[
    TestNumberOfColumnsWithMissingValues(),
    TestNumberOfRowsWithMissingValues(),
    TestNumberOfConstantColumns(),
    TestNumberOfDuplicatedRows(),
    TestNumberOfDuplicatedColumns(),
    TestColumnsType(),
    TestNumberOfDriftedColumns(),
])

tests.run(reference_data=reference, current_data=current)
tests
suite = TestSuite(tests=[
    NoTargetPerformanceTestPreset(),
])

suite.run(reference_data=reference, current_data=current)
suite

In [8]:
# reference['NAME_CONTRACT_TYPE'] = reference['NAME_CONTRACT_TYPE'].astype(str)
# Créez un ensemble de tests en utilisant les objets de colonne du DataFrame
suite = TestSuite(tests=[
#     TestColumnDrift(df_reference['NAME_CONTRACT_TYPE']),
    TestColumnDrift(column_name='target'),
    TestMeanInNSigmas('Age'),
    NoTargetPerformanceTestPreset(columns=['AMT_INCOME_TOTAL','AMT_CREDIT', 'CNT_CHILDREN'])
])


In [9]:
# Exécutez les tests
suite.run(reference_data=reference, current_data=current)
# 
suite

In [10]:

suite = TestSuite(tests=[
    TestNumberOfColumnsWithMissingValues(),
    TestNumberOfRowsWithMissingValues(),
    TestNumberOfConstantColumns(),
    TestNumberOfDuplicatedRows(),
    TestNumberOfDuplicatedColumns(),
    TestColumnsType(),
    TestNumberOfDriftedColumns(),
    TestColumnDrift('AMT_CREDIT'),
    TestShareOfOutRangeValues('AMT_CREDIT'),
    DataStabilityTestPreset(),
    RegressionTestPreset()    
])

In [11]:
suite.run(reference_data=reference, current_data=current)
suite


In [12]:
suite.as_dict()
suite.json()

'{"version": "0.4.8", "tests": [{"name": "The Number of Columns With Missing Values", "description": "The number of columns with missing values is 0. The test threshold is lte=0.", "status": "SUCCESS", "group": "data_integrity", "parameters": {"condition": {"lte": 0}, "value": 0.0}}, {"name": "The Number Of Rows With Missing Values", "description": "The number of rows with missing values is 0. The test threshold is lte=0 \\u00b1 1e-12.", "status": "SUCCESS", "group": "data_integrity", "parameters": {"condition": {"lte": {"value": 0.0, "relative": 0.1, "absolute": 1e-12}}, "value": 0.0}}, {"name": "Number of Constant Columns", "description": "The number of constant columns is 0. The test threshold is lte=0.", "status": "SUCCESS", "group": "data_integrity", "parameters": {"condition": {"lte": 0}, "value": 0.0}}, {"name": "Number of Duplicate Rows", "description": "The number of duplicate rows is 0. The test threshold is eq=0 \\u00b1 1e-12.", "status": "SUCCESS", "group": "data_integrity"

In [13]:
suite.save_html('test_suite_correct.html')

In [111]:
# Transformer toutes les colonnes en float
reference = reference.astype(float)

In [138]:
# ['SK_ID_CURR', 'TARGET', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT','Age']
# Importer la bibliothèque Pandas si ce n'est pas déjà fait
import pandas as pd

# Remplacer 'df' par le nom de votre DataFrame si ce n'est pas déjà fait
valeurs_uniques_r = reference['SK_ID_CURR'].unique()

# Afficher les valeurs uniques de la colonne 'target'
print(valeurs_uniques_r)
# Remplacer 'df' par le nom de votre DataFrame si ce n'est pas déjà fait
valeurs_uniques_c = current['SK_ID_CURR'].unique()

# Afficher les valeurs uniques de la colonne 'target'
print(valeurs_uniques_c)

[103521. 106666. 111628. ... 104630. 104221. 102305.]
[100304. 107046. 104517. ... 102224. 102952. 105431.]


In [148]:
import pandas as pd
import numpy as np

# Votre DataFrame
data = reference.copy()

# Vérification de la présence de NaN dans le DataFrame
has_nan = data.isnull().any().any()

# Vérification de la présence de valeurs non numériques dans le DataFrame
is_numeric = data.applymap(np.isreal).all().all()

if not has_nan and is_numeric:
    print("Toutes les colonnes contiennent des valeurs numériques et il n'y a pas de NaN.", has_nan,is_numeric)
else:
    print("Le DataFrame contient des valeurs non numériques ou des NaN.")


Toutes les colonnes contiennent des valeurs numériques et il n'y a pas de NaN. False True
